<strong>PyChemkin</strong> project can facilitate multiple <u>independent</u>  mechanisms. This tutorial demonstrates how to manage multiple <i>Chemistry Sets</i> (mechanisms) in a <strong>PyChemkin</strong> project. <strong>PyChemkin</strong> allows <u>only <em>ONE</em> active <i>Chemistry Set</i> at one time</u>. Once a new <i>Chemistry Set</i> is preprocessed, it becomes the <u>active</u> one, and the previously loaded <i>Chemistry Set</i> is deactivated. Any idle <i>Chemistry Set</i> must be <u>re-activated</u> before you can use it again in the project. 

This tutorial will load the first <i>Chemistry Set</i> <code>My1stMech</code> and perform some tasks with it. Then another <i>Chemistry Set</i> <code>My2ndMech</code> will be created and utilized. Afterwards, <code>My1stMech</code> will be re-activated and used to perform a different task.

In [ ]:
import os

import chemkin as ck  # Chemkin

# check working directory
current_dir = os.getcwd()
print("current working directory: " + current_dir)
# set verbose mode
ck.setverbose(True)

<h5>Set up the first <em>Chemistry Set</em></h5>The first task of every <strong>PyChemkin</strong> project is to create and preprocess the <em>Chemistry</em> set object for the current PyChemkin project. Here the GRI 3.0 mechanism for methane combustion is used to instantiate the <code>My1stMech</code> <em>Chemistry</em> object.

In [ ]:
# set mechanism directory (the default chemkin mechanism data directory)
data_dir = os.path.join(ck.ansys_dir, "reaction", "data")
mechanism_dir = data_dir
# set mechanism input files
# inclusion of the full file path is recommended
chemfile = os.path.join(mechanism_dir, "grimech30_chem.inp")
thermfile = os.path.join(mechanism_dir, "grimech30_thermo.dat")
tranfile = os.path.join(mechanism_dir, "grimech30_transport.dat")
# create a chemistry set based on GRI 3.0
My1stMech = ck.Chemistry(chem=chemfile, therm=thermfile, tran=tranfile, label="GRI 3.0")
# preprocess the mechanism files
iError = My1stMech.preprocess()
print()
if iError != 0:
    print(f"PreProcess: error encountered...code = {iError:d}")
    print(f"see the summary file {My1stMech.summaryfile} for details")
    exit()
else:
    print(ck.Color.GREEN + "PreProcess success!!", end="\n" + ck.Color.END)
    print("mechanism information:")
    print(f"number of elements = {My1stMech.MM:d}")
    print(f"number of gas species = {My1stMech.KK:d}")
    print(f"number of gas reactions = {My1stMech.IIGas:d}")

Once <code>My1stMech</code> is preprocessed successfully, it automatically becomes the <u>active</u> <i>Chemistry Set</i> in the project. 

<h5>Set up the first </em>Mixture</em></h5>First create the <code>mymixture1</code> mixture associated with <i>Chemistry Set</i> <code>My1stMech</code>. The properties of this mixture are arbitrary.

In [ ]:
# create a mixture with My1stMech
mymixture1 = ck.Mixture(My1stMech)
# set mixture temperature [K]
mymixture1.temperature = 1000.0
# set mixture pressure [dynes/cm2]
mymixture1.pressure = ck.Patm
# set molar compositions
mymixture1.X = [("CH4", 0.1), ("O2", 0.21), ("N2", 0.79)]

If you want to perform an equilibrium calculation but do not know the options, use the <code>help</code> method as given below. You will see finding equilibrium state with <u>"Specified H and P"</u> is option <u>#5</u>.  

In [ ]:
# compute the constrained H-P equilibrium state 
ck.help("equilibrium")

Perform the equilibrium calculation. The equilibrium state is stored as a <i>Mixture</i> <code>equil_mix1_HP</code>. You can get the equilibrium temperature by showing the <code>temperature</code> property of <i>Mixture</i> <code>equil_mix1_HP</code>. 

In [ ]:
equil_mix1_HP = ck.equilibrium(mymixture1, opt=5)
print(f"equilibrium temperature of mymixture1 : {equil_mix1_HP.temperature} [K]")

<h5>Set up the second <i>Chemistry Set</i></h5>Now you can load the second <i>Chemistry Set</i> <code>My2ndMech</code> based on the <i><u>C2_NOx_SRK.inp</u></i> mechanism file.

In [ ]:
#
# load the second mechanism
#
# set the 2nd mechanism directory (the default chemkin mechanism data directory)
mechanism_dir = data_dir
# create a chemistry set based on C2_NOx using an alternative method
My2ndMech = ck.Chemistry(label="C2 NOx")
# set mechanism input files individually
# this mechanism file contains all the necessary thermodynamic and transport data
# therefore no need to specify the therm and the tran data files
My2ndMech.chemfile = os.path.join(mechanism_dir, "C2_NOx_SRK.inp")
# instruct the preprocessor to include the transport properties
# only when the mechanism file contains all the transport data
My2ndMech.preprocesstransportdata()
# preprocess the 2nd mechanism files
iError = My2ndMech.preprocess()
print()
if iError != 0:
    print(f"PreProcess: error encountered...code = {iError:d}")
    print(f"see the summary file {My2ndMech.summaryfile} for details")
    exit()
else:
    print(ck.Color.GREEN + "PreProcess success!!", end="\n" + ck.Color.END)
    print("mechanism information:")
    print(f"number of elements = {My2ndMech.MM:d}")
    print(f"number of gas species = {My2ndMech.KK:d}")
    print(f"number of gas reactions = {My2ndMech.IIGas:d}")

<h5>Set up the second <em>Mixture</em></h5>Now <code>My2ndMech</code> is the <u>active</u> <i>Chemistry Set</i>, <code>My1stMech</code> is automatically de-activated. From this point on, you can work on tasks that involve <code>My2ndMech</code> <u>ONLY</u>. For example, you can create a <i>Mixture</i> <code>mymixture2</code> associated with <code>My2ndMech</code> and assign some arbitrary properties to <code>mymixture2</code>.

In [ ]:
# create the 2nd mixture with the My2ndMech
mymixture2 = ck.Mixture(My2ndMech)
# set mixture temperature [K]
mymixture2.temperature = 500.0
# set mixture pressure [dynes/cm2]
mymixture2.pressure = 2.0 * ck.Patm 
# set mixture molar composition
mymixture2.X = [("H2", 0.02), ("O2", 0.2), ("N2", 0.8)]

You can compute the C-J state and the detonation wave speed corresponding to the initial state given as <code>mymixture2</code>. 

In [ ]:
# compute detonation wave speed with mymixture2
speeds_mix2, CJ_mix2 = ck.detonation(mymixture2)
print(f"detonation mymixture2 temperature: {CJ_mix2.temperature} [K]") 
print(f"detonation wave speed = {speeds_mix2[1]/100.0} [m/sec]")

<h5>Re-activate the first <i>Chemistry Set</i></h5>Now you might curious about what is the detonation wave speed of <code>mymixture1</code>? Before performing the detonation calculation with <code>mymixture1</code> (which is associated with the inactive <code>My1stMech</code>), you have to re-activate the <i>Chemistry Set</i> <code>My1stMech</code>. Use the <code>active</code> method to switch <code>My1stMech</code> on.

In [ ]:
#
# re-activate My1stMech
My1stMech.activate()

Now you can perform the detonation wave speed calculation with <i>Mixture</i> <code>mymixture1</code>. 

In [ ]:
# compute detonation wave speed with mymixture1
speeds_mix1, CJ_mix1 = ck.detonation(mymixture1)
print(f"detonation mymixture1 temperature: {CJ_mix1.temperature} [K]") 
print(f"detonation wave speed = {speeds_mix1[1]/100.0} [m/sec]")

You can use the <code>active</code> method to turn on any idle <i>Chemistry Set</i> previously created in the project.